In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, ConcatDataset
from torch.utils.data import Dataset
from torch.utils.data import random_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pickle
import os
from PIL import Image

from mnistm import MNISTMDataset

In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.get_device_name()

'NVIDIA GeForce RTX 3080'

In [ ]:
# Define transformations
transform_mnist = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize to the same size as used in SVHN)
    transforms.Grayscale(num_output_channels=3),  # Convert to 3-channel RGB
    transforms.ToTensor(),  # Convert to Tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize (assuming grayscale, same value for all channels)
])

transform_mnist_m = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize for RGB
])

transform_svhn = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


In [ ]:
# Load datasets
mnist_train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform_mnist)
mnist_test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform_mnist)
#mnist_m_dataset = datasets.ImageFolder(root='./mnist_m', transform=transform_mnist_m)
mnistm_train_dataset = MNISTMDataset("./dataset", train = True, transform=transform_mnist_m)
mnistm_test_dataset = MNISTMDataset("./dataset", train = False, transform=transform_mnist_m)

svhn_train_dataset = datasets.SVHN(root='./data', split='train', download=True, transform=transform_svhn)
svhn_test_dataset = datasets.SVHN(root='./data', split='test', download=True, transform=transform_svhn)

Using downloaded and verified file: ./data\train_32x32.mat
Using downloaded and verified file: ./data\test_32x32.mat


In [ ]:
batch_size = 64  # Set the batch size

total_size = len(mnist_train_dataset)
val_size = int(0.15 * total_size)
train_size = total_size - val_size
train_dataset, val_dataset = random_split(mnist_train_dataset, [train_size, val_size])

mnist_train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
mnist_val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
mnist_test_loader = DataLoader(mnist_test_dataset, batch_size=batch_size, shuffle=False)

total_size = len(mnistm_train_dataset)
val_size = int(0.15 * total_size)
train_size = total_size - val_size
train_dataset, val_dataset = random_split(mnistm_train_dataset, [train_size, val_size])

mnistm_train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
mnistm_val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
mnistm_test_loader = DataLoader(mnistm_test_dataset, batch_size=batch_size, shuffle=False)

svhn_train_loader = DataLoader(svhn_train_dataset, batch_size=batch_size, shuffle=True)
svhn_test_loader = DataLoader(svhn_test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Feature extractor (e.g., a simple CNN for image tasks)
class FeatureExtractor(nn.Module):
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)  # Assuming input images are RGB (3 channels)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 32, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.drop = nn.Dropout2d()
        self.conv3 = nn.Conv2d(32, 64, 3)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 64, 3)
        # Calculate the size of the flattened features after the conv and pooling layers
        # After the pooling layers, a 32x32 image becomes 5x5
        self.fc1 = nn.Linear(64 * 5 * 5, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.bn1(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.drop(x)
        x = self.bn2(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = self.drop(x)
        x = x.view(-1, 64 * 5 * 5)  # Flatten the output
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x  # This feature vector is passed to the classifier

class Classifier(nn.Module):
    def __init__(self, num_classes=10):
        super(Classifier, self).__init__()
        # The input size should match the size of the feature vector from the feature extractor
        #self.fc1 = nn.Linear(84, 120)
        #self.fc2 = nn.Linear(120, 84)
        # The output size should match the number of classes in the classification task
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        #x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        x = self.fc3(x) # Logits for each class
        return F.softmax(x, dim=1)

In [ ]:
model = FeatureExtractor().to(device)

# Standard classification loss
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def train_model(model, train_loader, val_loader, test_loader, svhn_test_loader, num_epochs=10):
    # Send the model to the device (GPU or CPU)
    model.to(device)

    # Loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(num_epochs):
        print("For epoch: ", epoch)
        model.train()  # Set the model to training mode
        running_loss = 0.0
        correct = 0
        total = 0

        # Training loop
        for data in train_loader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Statistics
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_accuracy = correct / total
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")

        # Validation loop
        model.eval()  # Set the model to evaluation mode
        val_loss, val_accuracy = evaluate_model(model, val_loader, device, criterion)
        print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    # Test loop
    model.eval()
    test_loss, test_accuracy = evaluate_model(model, test_loader, device, criterion)
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy on Train Data Test: {test_accuracy:.4f}")
    test_loss, test_accuracy = evaluate_model(model, svhn_test_loader, device, criterion)
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy on Different Domain Data: {test_accuracy:.4f}")

def evaluate_model(model, loader, device, criterion):
    total = 0
    correct = 0
    running_loss = 0.0
    with torch.no_grad():  # No gradients needed for validation/testing
        for data in loader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    loss = running_loss / len(loader)
    accuracy = correct / total
    return loss, accuracy

In [ ]:
train_model(model, svhn_train_loader, svhn_test_loader, svhn_test_loader, mnist_test_loader, num_epochs = 10)

For epoch:  0
Epoch 1/10, Train Loss: 0.9094, Train Accuracy: 0.7051
Epoch 1/10, Validation Loss: 0.5028, Validation Accuracy: 0.8492
For epoch:  1
Epoch 2/10, Train Loss: 0.5029, Train Accuracy: 0.8499
Epoch 2/10, Validation Loss: 0.4226, Validation Accuracy: 0.8738
For epoch:  2
Epoch 3/10, Train Loss: 0.4464, Train Accuracy: 0.8677
Epoch 3/10, Validation Loss: 0.3878, Validation Accuracy: 0.8854
For epoch:  3
Epoch 4/10, Train Loss: 0.4160, Train Accuracy: 0.8747
Epoch 4/10, Validation Loss: 0.3556, Validation Accuracy: 0.8961
For epoch:  4
Epoch 5/10, Train Loss: 0.3907, Train Accuracy: 0.8835
Epoch 5/10, Validation Loss: 0.3430, Validation Accuracy: 0.8997
For epoch:  5
Epoch 6/10, Train Loss: 0.3769, Train Accuracy: 0.8876
Epoch 6/10, Validation Loss: 0.3301, Validation Accuracy: 0.9049
For epoch:  6
Epoch 7/10, Train Loss: 0.3577, Train Accuracy: 0.8929
Epoch 7/10, Validation Loss: 0.3290, Validation Accuracy: 0.9054
For epoch:  7
Epoch 8/10, Train Loss: 0.3496, Train Accuracy: 